In [ ]:
#Use conda env open2c_env.yml to create conda env for this script

#This is an example script of read normalization between samples for Hi-C datasets. 
#It is run from within the 'scripts' subdirectory, using following directory structure:
#Analysis_Dir
#├── data
#├── figures
#├── scripts
#├── lsf_jobs


In [ ]:
#Very useful examples of cooltools analysis and docs: https://cooltools.readthedocs.io/en/latest/notebooks/viz.html

# Import the packages we will use
#Utilities
import os
import re
import itertools
import glob
import pickle
import argparse

#Data Management
import numpy as np
from numpy import diff
import pandas as pd
import h5py
import scipy
from scipy.stats import linregress
from scipy import ndimage
from functools import partial
from scipy.linalg import toeplitz

#Plotting
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
import seaborn as sns
import upsetplot
from upsetplot import UpSet

#Genomics
import pairtools
import cooler
import cooltools
import bioframe
from bioframe import overlap

In [ ]:
#Using hg38 aligned files, sampling coolers to same number of interactions between samples/replicates

In [ ]:
#Set up the sample names/files/directories needed

dataDir = #'dir/to/mapped/mcools'
outDataDir = '..'

conditions = [
    'AS_DMSO_R1',
    'AS_ICRF_R1',
    'MR_t4DMSO_R1',
    'MR_t4ICRF_R1',
    'MR_t8DMSO_R1',
    'MR_t8ICRF_R1',
    'AS_DMSO_R2',
    'AS_ICRF_R2',
    'MR_t4DMSO_R2',
    'MR_t4ICRF_R2',
    'MR_t8DMSO_R2',
    'MR_t8ICRF_R2',
    'AS_DMSO_R1R2',
    'AS_ICRF_R1R2',
    'MR_t4DMSO_R1R2',
    'MR_t4ICRF_R1R2',
    'MR_t8DMSO_R1R2',
    'MR_t8ICRF_R1R2'
]

long_names = {
    'AS_DMSO_R1' : 'TI-HiC-Dpn-HeLa-G1Sort-DMSO-2hr-4-29-R1-T1',
    'AS_ICRF_R1' : 'TI-HiC-Dpn-HeLa-G1Sort-ICRF-2hr-4-29-R1-T1',
    'MR_t4DMSO_R1' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-DMSO-2hr-G1Sort-4-44-R1-T1',
    'MR_t4ICRF_R1' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-ICRF-2hr-G1Sort-4-44-R1-T1',
    'MR_t8DMSO_R1' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-DMSO-6hr-G1Sort-4-44-R1-T1',
    'MR_t8ICRF_R1' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-ICRF-6hr-G1Sort-4-44-R1-T1',
    'AS_DMSO_R2' : 'TI-HiC-Dpn-HeLa-G1Sort-DMSO-2hr-4-48-R2-T1',
    'AS_ICRF_R2' : 'TI-HiC-Dpn-HeLa-G1Sort-ICRF-2hr-4-48-R2-T1',
    'MR_t4DMSO_R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-DMSO-2hr-G1Sort-4-49-R2-T1',
    'MR_t4ICRF_R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-ICRF-2hr-G1Sort-4-49-R2-T1',
    'MR_t8DMSO_R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-DMSO-6hr-G1Sort-4-49-R2-T1',
    'MR_t8ICRF_R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-ICRF-6hr-G1Sort-4-49-R2-T1',
    'AS_DMSO_R1R2' : 'TI-HiC-Dpn-HeLa-G1Sort-DMSO-2hr-R1R2',
    'AS_ICRF_R1R2' : 'TI-HiC-Dpn-HeLa-G1Sort-ICRF-2hr-R1R2',
    'MR_t4DMSO_R1R2': 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-DMSO-2hr-G1Sort-R1R2',
    'MR_t4ICRF_R1R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t4hr-ICRF-2hr-G1Sort-R1R2',
    'MR_t8DMSO_R1R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-DMSO-6hr-G1Sort-R1R2',
    'MR_t8ICRF_R1R2' : 'TI-HiC-Dpn-HeLa-MitoticRelease-t8hr-ICRF-6hr-G1Sort-R1R2'
}

In [ ]:
ComboConds = [
    'AS_DMSO_R1R2',
    'AS_ICRF_R1R2',
    'MR_t4DMSO_R1R2',
    'MR_t4ICRF_R1R2',
    'MR_t8DMSO_R1R2',
    'MR_t8ICRF_R1R2',
]

SepConds = [
    'AS_DMSO_R1',
    'AS_ICRF_R1',
    'MR_t4DMSO_R1',
    'MR_t4ICRF_R1',
    'MR_t8DMSO_R1',
    'MR_t8ICRF_R1',
    'AS_DMSO_R2',
    'AS_ICRF_R2',
    'MR_t4DMSO_R2',
    'MR_t4ICRF_R2',
    'MR_t8DMSO_R2',
    'MR_t8ICRF_R2'
]

ComboCtrlConds = [
    'AS_DMSO_R1R2',
    'MR_t4DMSO_R1R2',
    'MR_t8DMSO_R1R2'
]

ComboTreatConds = [
    'AS_ICRF_R1R2',
    'MR_t4ICRF_R1R2',
    'MR_t8ICRF_R1R2'
]

SepCtrlConds = [
    'AS_DMSO_R1',
    'MR_t4DMSO_R1',
    'MR_t8DMSO_R1',
    'AS_DMSO_R2',
    'MR_t4DMSO_R2',
    'MR_t8DMSO_R2'
]

SepTreatConds = [
    'AS_ICRF_R1',
    'MR_t4ICRF_R1',
    'MR_t8ICRF_R1',
    'AS_ICRF_R2',
    'MR_t4ICRF_R2',
    'MR_t8ICRF_R2'
]

In [ ]:
#In shell, run multiQC on mapped data to discern the lowest read number in this set of samples
#conda activate multiQC-env
#multiqc -m pairtools #add_directory_to_distiller_results_here

In [ ]:
#Read in coolers - 10kb bins
binsize = 10000

clr_paths_10kb = {}
for cond in SepConds:
    clr_paths_10kb[cond] = f'{dataDir}/coolers_library/{long_names[cond]}.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
for cond in ComboConds:
    clr_paths_10kb[cond] = f'{dataDir}/coolers_library_group/{long_names[cond]}.hg38.mapq_30.1000.mcool::resolutions/{binsize}'
    
    
clrs10kb = {
    cond: cooler.Cooler(clr_paths_10kb[cond]) for cond in conditions
}

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38', as_bed=True)
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms = bioframe.split(hg38_chromsizes, hg38_cens, cols_points=['chrom', 'mid'])
# Select only chromosomes that are present in the cooler.
hg38_chromsizes = hg38_chromsizes.set_index("chrom").loc[clrs10kb['AS_DMSO_R1'].chromnames].reset_index()
hg38_arms = hg38_arms.set_index("chrom").loc[clrs10kb['AS_DMSO_R1'].chromnames].reset_index()
hg38_arms = bioframe.parse_regions(hg38_arms)

In [ ]:
print(bioframe.__version__)

In [ ]:
print(cooltools.__version__)

In [ ]:
print(cooler.__version__)

In [ ]:
#calculate read depth?
clrs10kb['MR_t8ICRF_R1'].info #Sum is the same as read # actually used in making cooler
#This is the file with the lowest read-depth from multiQC analysis above - sample the rest to this level

In [ ]:
#Sample for just 1000bp cooler (smallest) - then combine replicates and zoomify
sample_binsize = 1000

In [ ]:
clrPaths = {}
for cond in SepConds:
    clrPaths[cond] = f'{dataDir}/coolers_library/{long_names[cond]}.hg38.mapq_30.1000.mcool::resolutions/{sample_binsize}'


In [ ]:
#sample each to approx 16812305 in this case - has to be one less than the sum of the smallest cooler
#We use the bsub job scheduler - this can be modified based on your computing resources

for cond in SepConds:
    in_fname = clrPaths[cond]
    out_fname = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.cool'
    !bsub -q short -W 01:00 -e ../lsf_jobs/LSB_%J.err -o ../lsf_jobs/LSB_%J.log \
        -n 1 -R span[hosts=1] -R select[ib] -R rusage[mem=8000] -R select[rh=8] \
        "cooltools random-sample -c 16812305 --exact $in_fname $out_fname"


In [ ]:
sampledPaths = {}
for cond in SepConds:
    sampledPaths[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.cool'
for cond in ComboConds:
    sampledPaths[cond] = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.cool'

In [ ]:
#Merge replicates for combined files
for (cond1, cond2, combinedCond) in zip(conditions[0:6], conditions[6:12], ComboConds):
    
    cond1Sampled = sampledPaths[cond1]
    cond2Sampled = sampledPaths[cond2]

    combinedfile = sampledPaths[combinedCond]
    
    cooler.merge_coolers(combinedfile, [cond1Sampled, cond2Sampled], 20000000)

In [ ]:
conditions[0:6]

In [ ]:
conditions[6:12]

In [ ]:
ComboConds

In [ ]:
#Zoomify and balance sampled cooler files
for cond in conditions:
    coolfile = sampledPaths[cond]
    mcoolfile = f'{outDataDir}/data/{long_names[cond]}.sampled.hg38.mapq_30.1000.mcool'
    !bsub -q short -W 01:00 -e ../lsf_jobs/LSB_%J.err -o ../lsf_jobs/LSB_%J.log \
        -n 4 -R select[ib] -R span[hosts=1] -R rusage[mem=4000] -R select[rh=8] \
        "cooler zoomify -p 4 --balance --resolutions 4DN $coolfile -o $mcoolfile"

In [ ]:
#Please note that the exact interactions included in the sampled cooler files will be slightly different each time
#this is run, as they are produced by random sampling. To replicate the published figures, the exact processed 
#sampled .mcool files used in the published analyses are available on GEO. 